In [ ]:
# %load C:\Users\pandh\Desktop\Assignments\Visual Analytics\Assignment_2\template\A2_decisionTree\scripts\E3_T1andT2_igAndCategories.py

"""Code for exercise 3, task 1 and task 2."""

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

# scikit-learn for decision tree classification and evaluation
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

# load the data and make a copy with numerical entries only
df = pd.read_csv('../data/adult.csv', na_values='?').dropna()
df.drop(['fnlwgt'], axis=1, inplace=True)

datasetnum = df.copy()
datasetnum.drop(['education'], axis=1, inplace=True)

for i in list(datasetnum):
    if not (datasetnum[i].dtype == np.float64 or datasetnum[i].dtype == np.int64):
        datasetnum[i] = datasetnum[i].astype('category')
        datasetnum[i] = datasetnum[i].cat.codes


def IG(df, var):
    """
    Compute information gain for a given dataframe w.r.t. income 
    when splitting into categorical variable var.
    
    Return:
        float: information gain
    """
    # entropy before split
    entropy1 = df.income.value_counts(normalize=True).\
        apply(lambda x: -x*np.log2(x)).groupby(level=0).sum().sum()
    
    # entropy after split
    entropy = df.groupby([var]).income.value_counts(normalize=True).\
        apply(lambda x: -x*np.log2(x)).groupby(level=0).sum()
    weights = df[var].value_counts(normalize=True)
    entropy2 = entropy = (entropy*weights).sum()
    
    # return information gain
    return entropy1 - entropy2


var = 'race' # TODO edit here according to findings

# compute the information gain for one variable
print('Information gain for', var, IG(df, var), '\n')

# list all categories of a categorical variable
print(df[var].astype('category').cat.categories, '\n')

# TODO compute the information gain for your own splits
# all levels that are listed are in group, all others in the other
leftChild = ['Black', 'White']
df['split'] = df[var].apply(lambda x: x in leftChild)
 print('IG for %s %.6f' % (var, IG(df,'split')), 'group2: {"else"}', 'group1:', leftChild)
df.drop(['split'], axis=1, inplace=True)


# apply the sorting to the dataframe
dfSorted = df.copy()
dfSorted.drop(['education'], axis=1, inplace=True)

for i in list(dfSorted):
    if not (dfSorted[i].dtype == np.float64 or dfSorted[i].dtype == np.int64):
        # define a specific order for race
        if var in i: # TODO Adapt
            catType = CategoricalDtype(categories=[
                'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'
                ], ordered=True)
            dfSorted[i] = dfSorted[i].astype(catType)
        else:
            dfSorted[i] = dfSorted[i].astype('category')
        dfSorted[i] = dfSorted[i].cat.codes


In [2]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import graphviz
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

# Load and preprocess the data
df = pd.read_csv('../data/adult.csv', na_values='?').dropna()

# Print the code for education
display(df[['education', 'education-num']].drop_duplicates().sort_values(['education-num']))

# Drop unnecessary columns
df.drop(['fnlwgt', 'education'], axis=1, inplace=True)

# Convert categorical variables to numerical codes
for i in list(df):
    if not (df[i].dtype == np.float64 or df[i].dtype == np.int64):
        # Define a specific order for race
        if 'race' in i:
            cat_type = CategoricalDtype(categories=[
              'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'], ordered=True)
            df[i] = df[i].astype(cat_type)
        else:
            df[i] = df[i].astype('category')
        df[i] = df[i].cat.codes

# Select the data for modeling
datasub = df
X = datasub.iloc[:, 0:-1]
Y = datasub.iloc[:, -1]

# Train a decision tree
clf = tree.DecisionTreeClassifier(criterion="entropy", max_leaf_nodes=20)
clf = clf.fit(X, Y)

# Compute dot layout of tree
rawdot = export_graphviz(clf, out_file=None, feature_names=X.columns, 
                         class_names=['<=50k', '>50k'], filled=True)#, proportion=True)

# Scale the nodes
rawdot = rawdot[:15] + 'graph [size="10"];' + rawdot[15:]

# Render the tree
display(graphviz.Source(rawdot))

# Create a humorous decision tree based on the data
# This will extend the original funny tree with additional rules based on our model

funny_dot_code = '''
digraph FunnyTree {
    node [shape=box, style=filled, fontname=Helvetica, fontsize=10];
    
    // Root node
    "Start" [label="Do you intend to get married?", color=white];
    
    // First level decisions
    "MarriageYes" [label="Will you go to college?", style=filled, fillcolor="red"];
    "MarriageNo" [label="Your capital gain:", color=white];
    
    // Second level decisions
    "CollegeYes" [label="That's a great start! Keep going!", style=filled, fillcolor="red"];
    "CollegeNo" [label="Well, life might get tricky!", style=filled, fillcolor="red"];
    "CapitalGainLow" [label="Your savings could use a boost...", style=filled, fillcolor="red"];
    "CapitalGainHigh" [label="Your chances are high!", style=filled, fillcolor="green"];
    
    // Additional humorous branches
    "Job" [label="Did you land a high-paying job?", color=white];
    "JobYes" [label="Congrats! Your income will likely exceed 50K!", style=filled, fillcolor="green"];
    "JobNo" [label="You might want to explore some side hustles...", style=filled, fillcolor="red"];
    "SideHustles" [label="Are you considering a side hustle?", color=white];
    "SideHustleYes" [label="Great! This might help you reach your goal!", style=filled, fillcolor="green"];
    "SideHustleNo" [label="Consider living frugally.", style=filled, fillcolor="red"];
    
    // Connections between nodes
    "Start" -> "MarriageYes" [label="yes"];
    "Start" -> "MarriageNo" [label="no"];
    "MarriageYes" -> "CollegeYes" [label="yes"];
    "MarriageYes" -> "CollegeNo" [label="no"];
    "MarriageNo" -> "CapitalGainLow" [label="<=7000"];
    "MarriageNo" -> "CapitalGainHigh" [label=">7000"];
    "CapitalGainLow" -> "Job" [label="?"];
    "CapitalGainHigh" -> "JobYes" [label="High income job"];
    "CapitalGainLow" -> "JobNo" [label="Low income job"];
    "JobNo" -> "SideHustles" [label="look for options"];
    "SideHustles" -> "SideHustleYes" [label="yes"];
    "SideHustles" -> "SideHustleNo" [label="no"];
}

'''

# Render the humorous decision tree
display(graphviz.Source(funny_dot_code))

# To save the humorous decision tree as an image file, use:
# graphviz.Source(funny_dot_code).render('E3-figures/funny_decision_tree', format='png')


,education,education-num
224,Preschool,1
416,1st-4th,2
56,5th-6th,3
15,7th-8th,4
6,9th,5
219,10th,6
3,11th,7
415,12th,8
2,HS-grad,9
10,Some-college,10


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH